In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.set(font_scale=1.5)

In [ ]:
pd.options.display.max_columns=500
pd.options.display.max_rows=500
pd.options.display.max_colwidth=500


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/student-mat.csv',sep=';')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.school.unique()

In [ ]:
df.isnull().sum()

# Cleaning the outliers

In [ ]:

sns.boxplot(data = df, orient='h')

In [ ]:
def computeUpperFence (df_col, up=True):
  iqr = df_col.quantile(.75) - df_col.quantile(.25) # inter quartile range
  if up:
    return df_col.quantile(.75) + iqr*1.5
  return df_col.quantile(.25) - iqr*1.5

In [ ]:
sns.boxplot(data=df[['absences']], orient='h')

In [ ]:
# Checking the outliers
abs = df['absences']
abs_up_lim = computeUpperFence(abs)
df[ abs>abs_up_lim]
# They are atypical not outliers.

In [ ]:
sns.boxplot(data=df[['traveltime']], orient='h')

In [ ]:
# Checking the outliers
trav = df['traveltime']
trav_up_lim = computeUpperFence(trav)
df[ trav>trav_up_lim]


In [ ]:
sns.boxplot(data=df[['Fedu']], orient='h')

In [ ]:
# Checking the outliers
F = df['Fedu']
F_low_lim = computeUpperFence(F)
df[ F<F_low_lim]


In [ ]:
# lets check distribution of target variable

In [ ]:
df["G3"].hist(bins=25)
plt.xlabel("G3")
plt.ylabel("Count")
plt.show()
#it almost follow normal distribution

# Capping approach

In [ ]:
# Select only numeric columns
num = df.select_dtypes(include='number')
num.columns



In [ ]:
# Iterate over each numeric column
for col in num.columns:
    # Compute the upper and lower fences for the current column
    upper_fence = computeUpperFence(df[col])
    lower_fence = computeUpperFence(df[col], up=False)

    # Cap the outliers in the current column
    df.loc[df[col] > upper_fence, col] = upper_fence
    df.loc[df[col] < lower_fence, col] = lower_fence


In [ ]:
sns.boxplot(data = df, orient='h')
# we capped all the extreme values

# Regularising the categorical columns

In [ ]:
catcols = df.select_dtypes(include='object').head()
catcols

# Binary categorical columns

In [ ]:
binary_cols = catcols.iloc[:,-8:].columns.to_list()

In [ ]:
binary_cols

In [ ]:
def print_uniques(cols,df=df):
  for col in cols:
    list_unique = df[col].unique()
    list_unique.sort()
    print(col,':\n', list_unique)
    print('Number of unique categories:', len(list_unique))
    print('---------------------------------')

In [ ]:
print_uniques(binary_cols)

#Multi-Categorical columns

In [ ]:
catcols

In [ ]:
multiCat_cols = list(set(catcols)^set(binary_cols))
multiCat_cols


In [ ]:
print_uniques(multiCat_cols)

#Saving the cleaned data set

In [ ]:
df.to_csv('student_clean.csv',index=False)